In [7]:
import importlib
import random
import argparse
import configparser
import numpy as np
import networkx as nx
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_sparse
from torch import Tensor
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.optim as optim

from torch_geometric.utils import negative_sampling, to_networkx
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator


import networkx as nx
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import scipy
import math


from dataset_utils import node_feature_utils
from dataset_utils.node_feature_utils import *
import my_utils as utils

importlib.reload(utils)

<module 'my_utils' from '/li_zhengdao/github/GenerativeGNN/my_utils.py'>

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument("--class_cond", action="store_true", default=True)
args = parser.parse_args({})
print(args.class_cond)


True


# Test list extend()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

t = np.linspace(0, 1.5*np.pi, 2500)
y = np.sin(t**2)+np.random.random(2500)*.6
print(y.shape)
fig, axes = plt.subplots(1, 1, figsize=(10, 12), dpi=150)

df = pd.DataFrame(y)
# plt.plot(df[0], 'lightblue', df[0].rolling(10).mean(), 'b')
# plt.plot(df[0])
# plt.show()
axes.plot(df[0],'lightblue', df[0].rolling(20).mean(), 'r')
# plt.plot(df[0],'lightblue', df[0].rolling(2).mean(), 'r')
# plt.show()
# plt.plot(df[0],'lightblue', df[0].rolling(5).mean(), 'r')
# plt.show()
# plt.plot(df[0],'lightblue', df[0].rolling(10).mean(), 'r')
# plt.show()
# plt.plot(df[0],'lightblue', df[0].rolling(20).mean(), 'r')
plt.show()

In [ ]:
def downsampling(s:np.ndarray, sample_len=64):
    s_len = s.shape[0]
    if s_len <= sample_len:
        downsampled = np.pad(s, pad_width=(0, sample_len-s_len), mode='minimum')
        return np.arange(sample_len), downsampled
    
    indice = np.linspace(0, s_len-1, sample_len)
    int_idc = [int(i) for i in indice]
    downsampled = np.array([s[i] for i in int_idc])
    
    return int_idc, downsampled

x = np.arange(100)/10
sig = x**2 + 0.05*x**1.5

plt.figure()
plt.plot(sig)
int_idc, downsampled = downsampling(sig, sample_len=111)
plt.scatter(int_idc, downsampled, c='r', s=10)
plt.show()

In [ ]:
# plot log(x) <= x/a + log(a) - 1 

plt.figure()
x = np.linspace(0.1, 1, 20)
y = np.log(x)
a = list(np.linspace(0.2, 1, 5))
plt.plot(x, y, label='log x')
for aa in a:
    ya = x/aa + np.log(aa) - 1
    plt.plot(x, ya, label=f'a={aa}')
plt.legend()
plt.show()


In [ ]:
# Conjuagate function:

x = np.linspace(0.01, 2, 20)
fx = x * np.log(x)

y = np.linspace(0.01, 2, 10)


plt.figure()
plt.plot(x, fx)
for yy in y:
    yx = x*yy - fx
    plt.plot(x, yx)
# plt.show()

# plot y:
yy = np.linspace(0.01, 2, 20)

ys = []
for y in yy:
    ys.append(np.max(x*y - fx))
plt.plot(yy, ys)
plt.show()

# test A^2

In [ ]:
def norm_to_one(m):
    m[m>0]=1
    return m

def str2np(s):
    rows = s.split(',')
    
    row_len = len(rows)
    col_len = len(rows[0])
    
    res = np.ones(shape=(row_len, col_len))
    
    for i, r in enumerate(rows):
        for j, c in enumerate(r):
            res[i, j] = int(c)
    return res

def draw_g(adj):
    g = nx.from_numpy_array(adj)
    # pos = nx.nx_pydot.graphviz_layout(g1)
    nx.draw_circular(g, with_labels=True)

a = "0101,1010,0101,1010"

a = str2np(a)
I = np.eye(4)

aa = a + I

draw_g(a)

In [ ]:
g = nx.cycle_graph(3)
g1 = nx.Graph()

nx.set_node_attributes(g, [1,2,4], 'labels')

print(type(g), g.nodes[1]['labels'])
g.__class__()
print('2:', type(g), g.nodes[1]['labels'])


In [ ]:
a2 = a @ a
print('a:', a)
print('a*a2:', a*a2)

In [ ]:
a3 = a2_adj @ a

print(a3)

In [ ]:

a = "0110,1010,1100,0000"

a = str2np(a)
draw_g(a)
I = np.eye(4)
aa = a + I

D = np.diag(np.sum(a, axis=1))

print('D:', D)
a2 = a @ a
aa2 = aa @ aa


a2 = a @ a
print('a:', a)
print('aa:',aa)
print('a*a2:', a*a2)

In [ ]:

a = "0110,1000,1000,0000"

a = str2np(a)
draw_g(a)
I = np.eye(4)
aa = a + I

a2 = a @ a
a2_norm = norm_to_one(a2) - I

a2_adj = a2_norm + a
print(a2_adj)

a3 = a2_adj @ a

print(a3)

# $Cycle Count = A \odot A^2 - A$

In [ ]:
cyc = a * a2 - a
print(cyc)

In [5]:
class SmilesDataset:
    def __init__(self, dataset, smiles) -> None:
        self.dataset = dataset
        self.smiles = smiles

    def __getitem__(self, idx):
        return self.dataset[idx]
    
    def __len__(self):
        return len(self.dataset)

In [6]:
from torch_geometric.datasets import QM9, MoleculeNet

a = MoleculeNet('DATA', name='hiv')

smiles = [i.smiles for i in a]
sd = SmilesDataset(a, smiles=smiles)

41127
CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2
Data(edge_attr=[40, 3], edge_index=[2, 40], smiles="CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2", x=[19, 9], y=[1, 1])


In [12]:

import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
	
class ImbalancedDatasetSampler(WeightedRandomSampler):
    def __init__(self, targets, num_samples=None):
        self.num_samples = len(targets) if num_samples is None else num_samples

        class_sample_count = torch.unique(torch.tensor(targets), return_counts=True)[1]
        weight = 1. / class_sample_count.float()
        print('weight: ', weight)
        samples_weight = weight[targets]
        self.samples_weight = torch.FloatTensor(samples_weight)

        super().__init__(self.samples_weight, self.num_samples, replacement=True)

In [21]:
targets = [i%2 for i in range(2000)]
targets += [i%3+1 for i in range(1000)]
targets += [0 for i in range(10000)]

from collections import Counter

print(Counter(targets))

Counter({0: 11000, 1: 1334, 2: 333, 3: 333})


In [22]:
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader

sampler = ImbalancedDatasetSampler(targets)

class MyDataset(Dataset):
    def __init__(self, data) -> None:
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]
    

dataset = MyDataset(targets)
dataloader = DataLoader(dataset=dataset, sampler=sampler, batch_size=128)

for i in dataloader:
    ii = [j.item() for j in i]
    print(Counter(ii))
    

weight:  tensor([9.0909e-05, 7.4963e-04, 3.0030e-03, 3.0030e-03])
Counter({3: 33, 1: 32, 2: 32, 0: 31})
Counter({3: 40, 1: 32, 0: 31, 2: 25})
Counter({2: 41, 3: 31, 1: 30, 0: 26})
Counter({3: 36, 1: 35, 0: 31, 2: 26})
Counter({0: 36, 3: 35, 1: 29, 2: 28})
Counter({0: 36, 2: 34, 3: 29, 1: 29})
Counter({0: 36, 2: 32, 1: 32, 3: 28})
Counter({2: 37, 0: 32, 3: 32, 1: 27})
Counter({3: 40, 0: 33, 1: 32, 2: 23})
Counter({2: 42, 1: 32, 3: 31, 0: 23})
Counter({3: 38, 1: 34, 0: 30, 2: 26})
Counter({2: 36, 3: 34, 0: 31, 1: 27})
Counter({1: 40, 3: 36, 0: 31, 2: 21})
Counter({2: 39, 3: 30, 0: 30, 1: 29})
Counter({1: 34, 3: 33, 0: 31, 2: 30})
Counter({3: 35, 0: 34, 2: 34, 1: 25})
Counter({2: 47, 3: 30, 1: 26, 0: 25})
Counter({1: 38, 3: 33, 2: 31, 0: 26})
Counter({3: 36, 1: 35, 0: 31, 2: 26})
Counter({1: 35, 0: 34, 3: 33, 2: 26})
Counter({2: 38, 1: 35, 0: 30, 3: 25})
Counter({2: 37, 1: 34, 3: 32, 0: 25})
Counter({1: 35, 2: 32, 0: 31, 3: 30})
Counter({3: 39, 0: 34, 2: 34, 1: 21})
Counter({3: 41, 0: 32,

In [1]:
def test_tuple(a, *output):
    print(a)
    print(output)

In [3]:
test_tuple(1, *(2))

TypeError: test_tuple() argument after * must be an iterable, not int